In [ ]:
from IPython.display import clear_output 
clear_output()

In [ ]:
!pip install wget
!pip install nudenet

In [ ]:
# import os
# os.chdir('/content/drive/MyDrive/NLP_course/task_4')

In [ ]:
# os.path.isdir('/content/drive/MyDrive/NLP_course/task_4')

True

In [ ]:
# !rm -rf task_4

In [ ]:
# !cp -r /content/drive/MyDrive/NLP_course/task_4/blacklist_domains .

In [ ]:
import collections
import glob
import gzip
import json
import os
import re
import uuid
import csv
import shutil
import random
from PIL import Image
from bs4 import BeautifulSoup
import wget
from nudenet import NudeClassifier
nsfw_classifier = NudeClassifier()

In [ ]:

URL_BLACKLIST = set()
HEADER = ["uuid", "image_url", "local_path", "alt_text", "context", "url", "segment", "warc_file"]
PATHS_FILE = "https://commoncrawl.s3.amazonaws.com/crawl-data/CC-MAIN-2021-31/warc.paths.gz"
DOWNLOAD_PREFIX = "https://commoncrawl.s3.amazonaws.com/"

BLACKLIST_FOLDER = "blacklist_domains"
#  Fill back_list urls
BLACKLIST_FILES = os.listdir(BLACKLIST_FOLDER)
for domains_list in BLACKLIST_FILES:
    if not os.path.isfile(os.path.join(BLACKLIST_FOLDER, domains_list)) or \
            not domains_list.endswith('.txt'): continue
    with open(os.path.join(BLACKLIST_FOLDER, domains_list), 'r') as fd:
        lines = fd.readlines()
        for line in lines:
            URL_BLACKLIST.add(line.rstrip())

def process_html(url, content, directory_data = None):
    """Processes single html webpage and extracts instructions as tasks."""

    domain = url.split('://')[1].split('/')[0]
    soup = BeautifulSoup(content, 'html.parser')
    returnme = list()
    # Remove unnecessary tags which could exist in <ol>
    for s in soup.select('script'):
        s.extract()
    for s in soup.select('noscript'):
        s.extract()
    for s in soup.select('table'):
        s.extract()
    for s in soup.select('figure'):
        s.extract()

    if domain == 'www.lifewire.com':
        for s in soup.find_all('div', {'class': 'theme-experttiptip'}):
            s.extract()
        for s in soup.find_all('div', {'class': 'theme-experttipimportant'}):
            s.extract()

    # For specific websites, need fine tune the parser to remove (.extract()) some
    # unnecessary tags to clean up the result got from ol.get_text()
    # if domain == 'www.wikihow.com':
    #     for s in soup.select('span'):
    #         s.extract()

    ols = soup.find_all('ol')

    for _, ol in enumerate(ols):

        if ol.find_all('img'):
            spans_in_image = ol.get_text(' ', strip=True).split("\n")
            # Only take in context texts which are more than 10 words (change later to higher number if required).
            segment, warc_file_name, directory_prefix = "", "", ""
            if directory_data:
                directory_prefix, segment, warc_file_name = directory_data
            for image_tag in ol.find_all('img'):
                if image_tag['alt'] != "":
                    try:
                        image_url = image_tag["src"]
                        
                        # check if the image is hosted on web.
                        if not image_url.startswith("http"):
                            image_url = image_tag.get("longdesc")
                            if not image_url.startswith("http"):
                                continue
                        # Duplicate file already downloaded. (Like some thread where user profile pops in all the
                        # conversations we will not miss the context as we taken dump of all the spans for the first
                        # image.
                        if os.path.isfile(os.path.join(directory_prefix, image_url.split('/')[-1])):
                            continue
                        image_filename = image_url.split('/')[-1]
                        if not (image_filename.lower().endswith('.jpeg') or
                                image_filename.lower().endswith('.png') or
                                image_filename.lower().endswith('.jpg')):
                            continue
                        image_filename = wget.download(image_url, out=directory_prefix)

                        if ('.gif' in image_filename.lower()):
                            os.remove(image_filename)
                            continue
                        
                        # Add context filtering here.
                        # Check the context of the image and filter spans_in_image if the filter causes spans_in_image
                        # to be empty then continue to next image.

                        # Image filtering
                        with Image.open(image_filename) as im:
                            w, h = im.size
                            if w < 400 or h < 400:
                                os.remove(image_filename)
                                continue
    
                        # Remove NSFW pictures
                        scores = nsfw_classifier.classify(image_filename)
                        if scores[image_filename]['safe'] < 0.4:
                            continue
                        
                        alt_text = image_tag['alt']
                        # Check if it's english
                        # If caption is less than 15 characters skip.
                        # At least 15 words of context paragraph.
                        context = spans_in_image[0] + ','.join(spans_in_image[1:])
                        my_uuid = str(uuid.uuid4())
                        web_url = url
                        warc_segment = segment
                        warc_fn = warc_file_name
                        write_list = [my_uuid, image_url, image_filename, alt_text, context, web_url, warc_segment, warc_fn]
                        returnme.append(write_list)
                    except Exception as e:
                        os.remove(image_filename)
                        continue

    return returnme

def _is_valid(url, inst):
    url_words = re.compile(r'\w+').findall(url.lower())
    instruction_words = re.compile(r'\w+').findall(inst.lower())

    phone_set = {'android', 'phone', 'iphone'}
    click_set = {'tap', 'click'}

    return (set(url_words + instruction_words).intersection(phone_set) and
            set(instruction_words).intersection(click_set))

# DomainStatsIdx
COUNT_IN_WARC = 0
COUNT_IS_RESPONSE = 1
COUNT_HTML = 2
COUNT_HTML_HAS_INST = 3
COUNT_INST = 4

def _parse_one_page(lines, stats, domain_stats, download_data=None):
    """Parses one page in warc file.
  Args:
    lines: the lines of WARC content to parse, which should contain single web
      interaction info, such as a request or a response
    stats: dict of {string, int}, for reason of failure and count
    domain_stats: dict of {domain: [a, b, c, d, e]} which are the counts of
      different DomainStatsIdx items for each domain
  Returns:
    list of triple (url, instruction, html_content) for each instruction found.
  """
    if not lines:
        return []
    if lines[0].strip() != 'WARC/1.0':
        stats['Error_no_WARC/1.0_in_head'] += 1
        return []

    url = None
    warc_type = None
    section = 1
    html_lines = []
    for _, line in enumerate(lines):
        line = line.strip()
        if section < 3:
            if not line:
                section += 1
        if section == 1:
            if line.startswith('WARC-Type: '):
                warc_type = line[len('WARC-Type: '):].strip()
            if line.startswith('WARC-Target-URI: '):
                url = line[len('WARC-Target-URI: '):].strip()
                # Extract support.google.com from
                # https://support.google.com/news/publisher-center/answer/9603942
                domain = url.split('://')[1].split('/')[0]
                # Skip black listed domains
                # Includes 
                if domain in URL_BLACKLIST:
                    return []
                # skip blacklisted urls (NSFW)
                # if FLAGS.filter_domain:
                #     if domain not in URL_WHITE_LIST:
                #         stats['NotFound_Domain_mismatch'] += 1
                #         return []
                domain_stats['DOMAIN_' + domain][COUNT_IN_WARC] += 1
                if warc_type == 'response':
                    domain_stats['DOMAIN_' + domain][COUNT_IS_RESPONSE] += 1

        if section == 3 and line:  # section 3 is html:
            html_lines.append(line)
    if not url or not html_lines:
        stats['No_HTML'] += 1
        return []

    domain_stats['DOMAIN_' + domain][COUNT_HTML] += 1

    try:
        html_content = '\n'.join(html_lines)
        instructions = process_html(url, html_content, download_data)
    except Exception:  # pylint: disable=broad-except
        stats['Error_parse_html'] += 1
        return []

    if not instructions:
        stats['No_instruction'] += 1
        return []

    stats['Got'] += 1
    domain_stats['DOMAIN_' + domain][COUNT_HTML_HAS_INST] += 1
    domain_stats['DOMAIN_' + domain][COUNT_INST] += len(
        instructions)
    return instructions

def extract_instructions_from_warc_file(warc_file_path, file_handler, download_data=""):
    """Reads instruction from WARC file.
  Args:
    warc_file_path: warc file path.
    file_handler: file handler of the warc file.
  Yields:
    triple(url, index, instruction)
  """
    lines_of_one_page = []
    stats = collections.defaultdict(int)
    domain_stats = collections.defaultdict(lambda: [0, 0, 0, 0, 0])

    for line in file_handler:
        if line.strip().startswith('WARC/1.0'):
            stats['Total'] += 1
            urls_and_instructions = _parse_one_page(lines_of_one_page,
                                                    stats, domain_stats, download_data=download_data)
            for csv_row in urls_and_instructions:
                yield csv_row
            lines_of_one_page = [line]
        else:
            lines_of_one_page.append(line)

    urls_and_instructions = _parse_one_page(lines_of_one_page,
                                            stats, domain_stats, download_data=download_data)
    stats['file_name'] = warc_file_path

    # if FLAGS.filter_domain:  # without filter, the log will be too long
    #     logging.info(json.dumps({**stats, **domain_stats}))
    for csv_row in urls_and_instructions:
        yield csv_row


In [ ]:
def fill_csv_download_image():
    # This is for the downloaded WARC files if they are stored in local device.
    # If the downloaded WARC files are stored in your own remote file system,
    # please costomize this part.

    segments = ['1627046152156.49', '1627046153531.10', '1627046153860.57', '1627046154032.75']
    warc_file = wget.download(PATHS_FILE)
    with gzip.open(warc_file, 'rb') as f_in:
        # Ignore .gz extention
        with open(warc_file[:-3], 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    segment_dict = collections.defaultdict(list)
    with open(warc_file[:-3], 'r') as warc_paths:
        for warc_line in warc_paths.readlines():
            for segment in segments:
                if segment in warc_line.rstrip():
                    segment_dict[segment].append(warc_line.rstrip())
                    break

    os.remove(warc_file)
    os.remove(warc_file[:-3])

    # # Taking 2 from each segment
    # for segment_k in segment_dict:
    #     segment_dict[segment_k] = random.sample(segment_dict[segment_k], 4)

    csv_file_descriptor = open("dataset4_task.csv", 'a', newline='')
    csv_writer = csv.writer(csv_file_descriptor)
    csv_writer.writerow(HEADER)
    try:
        for segment in segment_dict:
            for warc_file_download_link in segment_dict[segment]:
                warc_file = wget.download(DOWNLOAD_PREFIX + warc_file_download_link)
                download_dir = segment + "_" + warc_file.split(".")[0].split("-")[-1]
                os.makedirs(download_dir)
                with open(warc_file, 'rb') as f1:
                    with gzip.open(f1, mode='rt', encoding='latin1') as f2:
                        for csv_row in extract_instructions_from_warc_file(warc_file, f2, download_data=(download_dir, segment, warc_file)):
                            csv_writer.writerow(csv_row)
                os.remove(warc_file)

    finally:
        csv_file_descriptor.close()


In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
# os.chdir('/content/drive/MyDrive/NLP_course/task_4/')

In [ ]:
# We are downloading images locally as there was an issue with google colab,
# It got stuck randomly during a download.

In [ ]:
!pip install langdetect
!pip install -U sentence-transformers
!pip install pyyaml==5.1
!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu111/torch1.9/index.html

     |████████████████████████████████| 981 kB 8.8 MB/s 
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=f76262a502870ca65cdffc9fcb04490bc41e381d7ad0c9273d97be42dc04a2ea
  Stored in directory: /root/.cache/pip/wheels/c5/96/8a/f90c59ed25d75e50a8c10a1b1c2d4c402e4dacfa87f3aff36a
Successfully built langdetect
     |████████████████████████████████| 78 kB 4.6 MB/s 
     |████████████████████████████████| 2.9 MB 15.6 MB/s 
     |████████████████████████████████| 3.3 MB 55.7 MB/s 
     |████████████████████████████████| 1.2 MB 73.1 MB/s 
     |████████████████████████████████| 56 kB 6.5 MB/s 
     |████████████████████████████████| 596 kB 67.4 MB/s 
     |████████████████████████████████| 895 kB 73.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.1.0-py3-none-any.whl size=121000 sha256=83a77fccec9efbda647d30fa6268223723260598984b7753f5f0402510044c63
  Stored in directory: /root/.cache/pip/wheels/90/f0/bb/ed1add84

In [ ]:
from langdetect import detect
import scipy
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.95k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/399 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def _replace_unicode_with_space(text):
    """Replaces all unwanted unicode chars with single space."""
    returnme = ''.join([i if ord(i) < 128 else ' ' for i in text])
    returnme = ' '.join(returnme.split())  # Change all space/newline to one space
    return returnme

In [ ]:
import sys
import csv

csv.field_size_limit(sys.maxsize)

131072

In [ ]:
os.listdir()

['blacklist_domains',
 'dataset4_task.csv',
 '1627046153860.57_00197',
 '1627046153860.57_00001',
 '1627046153531.10_00422',
 '1627046153531.10_00390',
 '1627046153531.10_00311',
 '1627046153531.10_00220',
 '1627046153531.10_00146',
 '1627046152156.49_00630',
 '1627046152156.49_00577',
 '1627046152156.49_00505',
 '1627046152156.49_00458',
 '1627046152156.49_00170',
 'filtered_dataset4_task.csv']

In [ ]:
# [my_uuid, image_url, image_filename, alt_text, context, web_url, warc_segment, warc_fn]
HEADER = ["uuid", "image_url", "local_path", "alt_text", "context", "url", "segment", "warc_file"]
import csv
filtered_csv_file_descriptor = open("filtered_dataset4_task.csv", 'w', newline='')
filtered_csv_writer = csv.writer(filtered_csv_file_descriptor)
filtered_csv_writer.writerow(HEADER)
count_non_english = 0

def fix_nulls(s):
    for line in s:
        yield line.replace('\0', '')

with open("dataset4_task.csv", 'r', newline='') as csv_file_descriptor:
    csv_reader = csv.reader(fix_nulls(csv_file_descriptor))
    next(csv_reader, None)  # skip the headers
    for [my_uuid, image_url, image_filename, alt_text, context, web_url, warc_segment, warc_fn] in csv_reader:
        # valid picture filtering, also removes .tmp or incomplete downloads
        if not os.path.exists(image_filename): continue
        if not (image_filename.lower().endswith('jpg') or 
                image_filename.lower().endswith('jpeg') or
                image_filename.lower().endswith('png')):
                os.remove(image_filename)
                continue

        alt_text = _replace_unicode_with_space(alt_text)
        context = _replace_unicode_with_space(context)
        try:
            detect(alt_text) != 'en'
            detect(context) != 'en'
            # We replaced all non unicode, just checkif the string is still valid.
            # Detect throws exception when the string is not a valid language.
            # follwing bert model supports more than 80 languages so no need to
            # ignore languages like france, spanish etc.
        except Exception:
            # No letters just symbols
            count_non_english += 1
            if count_non_english <= 10:
                print(f"NON ENGLISH OR EMPTY ALT_TEXT / CONTEXT \nALTTEXT: {alt_text}\nContext: {context}")
            os.remove(image_filename)
            continue
        
        # Nsfw image filtering is taken care by nude net already.
        # Filter context, in the dumped texts check for atmost 5 sentences which relate.
        context = [i for i in context.split('\n')]
        new_context = list()
        for i in context:
            if len(i.split(' ')) > 4:
                new_context.append(i)
        context = new_context
        if len(context) == 0:
            os.remove(image_filename)
            continue
        alt_emb = model.encode(alt_text)
        context_emb = model.encode(context)
        if not context_emb.any():
            continue
        dist = scipy.spatial.distance.cdist([alt_emb], context_emb, "cosine")
        results = zip(range(len(dist)), dist)
        results = sorted(results, key=lambda x: x[1])
        # Only have top 5 sentence
        new_context = [context[idx] for idx, dist in results[:5]]
        context = new_context[0] + '\n'.join(new_context[1:])
        
        filtered_csv_writer.writerow([my_uuid, image_url, image_filename, alt_text, context, web_url, warc_segment, warc_fn])
filtered_csv_file_descriptor.close()

print("________________________________________________________________________")
print(f"STATS: Non-english alt-text or context skipped: {count_non_english}")
print("________________________________________________________________________")

NON ENGLISH OR EMPTY ALT_TEXT / CONTEXT 
ALTTEXT: 450 (450 / )
Context: Fiio M6 Titanium Fiio M6 Titanium : M6 Titanium : Fiio FiiO FH5 titanium FiiO FH5 titanium : FH5 titanium : FiiO 450 (450 / ) 450 (450 / ) : 450 (450 / ) : GEOZON TITANIUM Blue GEOZON TITANIUM Blue : Titanium G-SM10BLKB : Geozon Soundcare SuperSpike SA Titanium ( 3 .) Soundcare SuperSpike SA Titanium ( 3 .) : SuperSpike SA Titanium ( 3 .) : Soundcare Gamma Piu Extra Pro XL LCD Titanium Gamma Piu Extra Pro XL LCD Titanium : Extra Pro XL LCD Titanium : Gamma Piu Ubiquiti Bullet M5 Titanium BM5-TI Ubiquiti Bullet M5 Titanium BM5-TI : Bullet M5 Titanium : Ubiquiti AfterShokz Trekz Titanium Ocean-Blue AS600OB AfterShokz Trekz Titanium Ocean-Blue AS600OB : Trekz Titanium : AfterShokz DFC World Cup DFC World Cup : World Cup : DFC 450 Diolex (DXMV-450-2) 450 Diolex (DXMV-450-2) : (DXMV-450-2) : Diolex Royal Cup 450ml BK-77 Royal Cup 450ml BK-77 : BK-77 : Royal Cup 0.4 Walmer Eco Cup (W24201801) 0.4 Walmer Eco Cup (W2420180

In [ ]:
import torch
torch.__version__

'1.9.0+cu111'

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [ ]:
# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
image_path_list = list()
my_display_images = [
'1627046152156.49_00577/1627236237_959426_1627240259_miniatura_normal.jpg',
'1627046152156.49_00170/must-try-food-and-drink-destinations-outside-the-gta.jpg',
'1627046152156.49_00458/107-2-1.jpg',
'1627046152156.49_00458/107-5-1.jpg',
'1627046152156.49_00458/107-2-5.jpg',
'1627046152156.49_00458/107-2-6.jpg',
'1627046152156.49_00577/30-0482-1-A_8.jpg',
'1627046152156.49_00577/wGR150_MODEL_WATER_ANGLE.jpg',
'1627046152156.49_00577/35765LF-PD_MODEL_02_WEB.jpg',
'1627046152156.49_00577/Interior-Design-KBIS-2020-Native_Trails_Positano_Shoreline_WEB_2000.jpg',
'1627046152156.49_00505/3441_white_limed_oak.jpg',
'1627046152156.49_00505/atlantic-slate_6.jpg',
'1627046152156.49_00505/Colonia_Wood_Nordic_White_Oak_4436_4.jpg',
'1627046152156.49_00505/3439_hickory_oak.jpg',
'1627046152156.49_00505/031669_675_01.jpg',
'1627046152156.49_00458/3xccj1ad0_911_f_1_4.jpg',
'1627046152156.49_00458/instinkt_huhn_napf_2.png',
'1627046152156.49_00458/KP_SA_16.jpg',
]
with open("filtered_dataset4_task.csv", 'r', newline='') as csv_file_descriptor:
    csv_reader = csv.reader(csv_file_descriptor)
    next(csv_reader, None)  # skip the headers
    for [my_uuid, image_url, image_filename, alt_text, context, web_url, warc_segment, warc_fn] in csv_reader:
        if image_filename in my_display_images:
            image_path_list.append((image_filename, alt_text, context))
        # Use selected images to sample the data
        # image_path_list.append((image_filename, alt_text, context))

In [ ]:
for data_instance in image_path_list:
    im = cv2.imread(data_instance[0])
    cfg = get_cfg()
    # add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
    cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
    cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
    # Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
    cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
    predictor = DefaultPredictor(cfg)
    outputs = predictor(im)
    v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
    out = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    print('\n')
    print('IMAGE PATH: ', data_instance[0])
    print('ALT TEXT: ', data_instance[1])
    print('CONTEXT: ', data_instance[2])
    cv2_imshow(out.get_image()[:, :, ::-1])

In [ ]:
#1627046152156.49_00348